# ROS2 Documentation for Beginners
## Chapter 3 Supplement of the ROS2 Communication Mechanism

Chapter 2 introduces the core content of the ROS 2 communication mechanism, focusing more on a high-level overview of the framework. This chapter provides supplementary details about the communication mechanism, such as building a distributed framework, handling naming conflicts, commonly used APIs, and tools related to the communication mechanism. These supplementary topics are somewhat scattered but not complex. Additionally, at the end of this chapter, several exercises are included to reinforce the understanding of the ROS 2 communication mechanism.communication mechanism in ROS 2.


### Overview of this chapter ### 

| Chapter | Learning Content | Learning Outcome |
| ------- | ---------------- | ---------------- |
| 3.1 Distributed | Concept of distributed communication, application scenarios, and communication rules. | Able to independently set up a distributed communication framework. |
| 3.2 Workspace Overlay | Issues that may arise when package names conflict across different workspaces. | Strive to avoid package name conflicts in the future. |
| 3.3 Meta Packages | Concept, application scenarios, and creation rules for meta packages. | Able to customize meta packages. |
| 3.4 Node Naming Conflicts | Issues and solutions for node naming conflicts. | Able to resolve node naming conflicts appropriately. |
| 3.5 Topic Naming Conflicts | Issues and solutions for topic naming conflicts. | Able to resolve topic naming conflicts appropriately. |
| 3.6 Time-Related APIs | Time-related APIs, such as `Rate`, `Time`, and `Duration`. | Able to use time-related APIs. |
| 3.7 Communication Tools | Common command-line tools and the `rqt` toolbox for communication. | Able to use communication tools to improve program development and debugging efficiency. |
| 3.8 Communication Mechanism Practice | Exercises related to various communication mechanisms. | Reinforces understanding of the ROS 2 communication mechanism. |
| 3.9 Chapter Summary | Summary of key points. | Review of key concepts. |



## 3.1 Data Distribution Se Communication


In ROS 2, although there are various communication methods, the components of each method are similar. For example: communication involves two or more parties, different communication objects need to be linked, each has its own model, and data exchange always requires a data carrier. This section will introduce some of the terminology involved in ROS 2 communication.


**Scenario**
In many robotics-related application scenarios, multiple ROS 2 devices work together, such as in autonomous vehicle fleets, drone swarms, and remote control operations. But how do different ROS 2 devices communicate with each other?

**Concept**
Distributed communication refers to a communication strategy that enables data exchange across different hosts over a network.

ROS 2 itself is a distributed communication framework, making it easy to achieve communication between different devices. ROS 2 is based on DDS (Data Distribution Service), and when devices are on the same network, distributed communication can be achieved through DDS's Domain ID mechanism (ROS_DOMAIN_ID). The basic process is as follows: before launching a node, you can set the Domain ID value. Nodes with the same Domain ID can discover and communicate with each other, while nodes with different Domain IDs cannot. By default, all nodes use Domain ID 0 when launched, meaning that as long as devices are on the same network, no additional configuration is needed for nodes on different ROS 2 devices to achieve distributed communication.

**Application**
Distributed communication is widely applicable. As mentioned above, in robotic swarms, robots may need to obtain information about the speed, position, and trajectory of neighboring robots. In remote control scenarios, the control unit may need to access environmental information collected by the robot and send control commands. All of these data exchanges rely on distributed communication.

**Implementation**
For multi-machine communication, you can group nodes by Domain ID. Nodes within the same group can communicate freely, while nodes in different groups cannot. If all nodes belong to the same group, the default Domain ID can be used. To divide nodes into multiple groups, you can set the Domain ID of a node before launching it in the terminal (e.g., set it to 6) using the following command:

```bash
export ROS_DOMAIN_ID=6
```

After executing this command, the node will be assigned to the domain with ID 6.

To set a uniform Domain ID for all nodes on the current device, use the following command:

```bash

echo "export ROS_DOMAIN_ID=6" >> ~/.bashrc
```
After completing this command, restart the terminal, and all nodes that are launched will automatically be assigned to the domain with ID 6.


**Note**
There are certain constraints when setting the value of `ROS_DOMAIN_ID`:

- It is recommended to set `ROS_DOMAIN_ID` within the range [0, 101], inclusive of 0 and 101.
- The total number of nodes within each Domain ID is limited to 120 or fewer.
- If the Domain ID is set to 101, the total number of nodes in that domain must be 54 or fewer.

**DDS Domain ID Calculation Rules**

The calculation rules for the Domain ID values in DDS are as follows:

- DDS communication is based on the TCP/IP or UDP/IP network protocols, which require specifying a port number. The port number is an unsigned 2-byte integer with a range of [0, 65535].
- Port allocation follows specific rules and cannot be arbitrarily assigned. According to the DDS protocol, 7400 is the starting port, making the usable port range [7400, 65535]. Additionally, by default, each Domain ID uses 250 ports, so the total number of possible Domain IDs is `(65535 - 7400) / 250 = 232`, meaning the Domain ID range is [0, 231].
- Operating systems also reserve certain ports. When using ports in DDS, these reserved ports should be avoided to prevent conflicts. Different operating systems have different reserved ports. As a result, on Linux, the usable Domain IDs are [0, 101] and [215, 231], while on Windows and macOS, the usable Domain IDs are [0, 166]. Therefore, for cross-platform compatibility, it is recommended to set Domain IDs within the range [0, 101].
- Each Domain ID defaults to using 250 ports, and each ROS 2 node occupies two ports. According to the DDS protocol, within each Domain ID's port range, the first and second ports are the Discovery Multicast and User Multicast ports. The Discovery Unicast and User Unicast ports for the first node in the domain start at the 11th and 12th ports, respectively. Ports for subsequent nodes are assigned sequentially. Thus, the maximum number of nodes per Domain ID is `(250 - 10) / 2 = 120`.
- Special Case: When the Domain ID is set to 101, the second half of the port range falls within the operating system’s reserved ports, limiting the maximum number of nodes to 54.

These calculation rules are primarily for reference.

```
Domain ID:      0
Participant ID: 0

Discovery Multicast Port: 7400
User Multicast Port:      7401
Discovery Unicast Port:   7410
User Unicast Port:        7411

---

Domain ID:      1
Participant ID: 2
Discovery Multicast Port: 7650
User Multicast Port:      7651
Discovery Unicast Port:   7664
User Unicast Port:        7665
```

## 3.2 MetaPackage 
**Scenario**

Completing a system-level feature may involve multiple packages. For example, implementing a robot navigation module could include different sub-packages for mapping, localization, path planning, etc. When a user installs this module, do they need to install each package individually?

Clearly, installing packages one by one is inefficient. In ROS 2, there is a way to bundle different packages into a single package. When installing a module, users can simply install the bundled package, known as a metapackage.


**Concepts**

A **Metapackage** is a concept from Linux file management systems. In ROS 2, it is a "virtual package" that contains no actual content but depends on other packages. This allows packages to be grouped together, functioning like an index in a book, listing the sub-packages included and where to download them.

For example:

When you use the command `sudo apt install ros-<ros2-distro>-desktop` to install ROS 2, a metapackage is used. This metapackage depends on other packages in ROS 2, so installing it will also install its dependencies.


**Purpose**

Metapackages make installation convenient for users. With a single package, all related packages are grouped together and installed in one go.

**Implementation**
- create a package. 
  ``` ros2 pkg create tutorials_plumbing ```
- Edit the package.xml
  ```xml
  <?xml version="1.0"?>
  <?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
  <package format="3">
      <name>tutorails_plumbing</name>
      <version>0.0.0</version>
      <description>TODO: Package description</description>
      <maintainer email="ros2@todo.todo">ros2</maintainer>
      <license>TODO: License declaration</license>

      <buildtool_depend>ament_cmake</buildtool_depend>

      <exec_depend>base_interfaces_demo</exec_depend>
      <exec_depend>cpp01_topic</exec_depend>
      <exec_depend>cpp02_service</exec_depend>
      <exec_depend>cpp03_action</exec_depend>
      <exec_depend>cpp04_param</exec_depend>
      <exec_depend>py01_topic</exec_depend>
      <exec_depend>py02_service</exec_depend>
      <exec_depend>py03_action</exec_depend>
      <exec_depend>py04_param</exec_depend>


      <export>
        <build_type>ament_cmake</build_type>
      </export>
    </package>
```

- Edit the CMakeLists.txt
  ```cmake
      cmake_minimum_required(VERSION 3.8)
      project(tutorails_plumbing)

      if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
        add_compile_options(-Wall -Wextra -Wpedantic)
      endif()

      find_package(ament_cmake REQUIRED)

      ament_package()
```

### 3.3 Time related APIs

In previous exmaples, we used the `Rate` class to control the frequency of message publishing. In this section, we will introduce other time-related APIs in ROS 2.

<img src="figs/excution_demo.gif" alt="Execution Demo" width="300" autoplay>


#### 3.3.1 Rate Class ####

In the topic communication example in Chapter 2, the topic publisher is required to publish messages at a certain frequency. In our implementation, we use a timer to control the publishing rate. However, besides timers, ROS 2 also provides the `Rate` class, which can also be used to control the execution frequency of a program.

**Rate Class in rclpy**
Example: Display a text periodically at a fixed frequency.

```python

import rclpy
import threading
from rclpy.timer import Rate

rate = None
node = None

def do_some():
    global rate
    global node
    while rclpy.ok():
        node.get_logger().info("hello ---------")
        # Sleep
        rate.sleep()

def main():
    global rate
    global node
    rclpy.init()    
    node = rclpy.create_node("rate_demo")
    # Create Rate Object
    rate = node.create_rate(1.0)

    # Create sub-thread
    thread = threading.Thread(target=do_some)
    thread.start()

    rclpy.shutdown()

if __name__ == "__main__":
    main()

```
#### 3.3.2 Time ####
Example: Create Time function.

```python
import rclpy
from rclpy.time import Time
def main():
    rclpy.init()
    node = rclpy.create_node("time_demo")
    # Create Time Object
    right_now = node.get_clock().now()
    t1 = Time(seconds=10,nanoseconds=500000000)

    node.get_logger().info("s = %.2f, ns = %d" % (right_now.seconds_nanoseconds()[0], right_now.seconds_nanoseconds()[1]))
    node.get_logger().info("s = %.2f, ns = %d" % (t1.seconds_nanoseconds()[0], t1.seconds_nanoseconds()[1]))
    node.get_logger().info("ns = %d" % right_now.nanoseconds)
    node.get_logger().info("ns = %d" % t1.nanoseconds)
    rclpy.shutdown()

if __name__ == "__main__":
    main()

```
#### 3.3.3 Duration Class ####
Create a Duration object.

```python

import rclpy
from rclpy.duration import Duration

def main():
    rclpy.init()    

    node = rclpy.create_node("duration_demo")
    du1 = Duration(seconds = 2,nanoseconds = 500000000)
    node.get_logger().info("ns = %d" % du1.nanoseconds)

    rclpy.shutdown()

if __name__ == "__main__":

    main()

```
```python
import rclpy
from rclpy.time import Time
from rclpy.duration import Duration

def main():
    rclpy.init()
    node = rclpy.create_node("time_opt_node")
    t1 = Time(seconds=10)
    t2 = Time(seconds=4)

    du1 = Duration(seconds=3)
    du2 = Duration(seconds=5)

    # Compare
    node.get_logger().info("t1 >= t2 ? %d" % (t1 >= t2))
    node.get_logger().info("t1 < t2 ? %d" % (t1 < t2))
    # Compute
    t3 = t1 + du1
    t4 = t1 - t2    
    t5 = t1 - du1

    node.get_logger().info("t3 = %d" % t3.nanoseconds)
    node.get_logger().info("t4 = %d" % t4.nanoseconds)
    node.get_logger().info("t5 = %d" % t5.nanoseconds)

    # Compare
    node.get_logger().info("-" * 80)
    node.get_logger().info("du1 >= du2 ? %d" % (du1 >= du2))
    node.get_logger().info("du1 < du2 ? %d" % (du1 < du2))

    rclpy.shutdown()

if __name__ == "__main__":
    main()
```


### 3.3 Communication Tools ###

**Scenario**  
In the previous chapter, we explored various communication mechanisms in ROS 2, understanding the implementation process, related APIs, and characteristics of each communication model. Now, we will discuss some practical issues that may arise during development:

1. After launching a complete robot system, the setup can be quite complex, possibly consisting of dozens or even hundreds of nodes. Each node might contain one or more communication entities (such as topic publishers, topic subscribers, servers, clients, action servers, action clients, parameter servers, and parameter clients). Additionally, various message (`msg`), service (`srv`), or action interfaces are used during communication. So, how can developers easily access information about these nodes, topics, services, actions, parameters, and interfaces?
2. When implementing communication, at least two parties are involved. Once one side is completed, how can we verify if the program functions correctly?
3. During topic communication, a publishing frequency is set in the publisher’s code. How can we verify if the actual publishing frequency matches the specified frequency during runtime?
   
ROS 2 provides tools that efficiently address these questions. This section introduces these tools and how to use them.

**Concept**  
In ROS 2, tools related to communication mechanisms fall into two categories: command-line tools and graphical tools (rqt). The command-line tools are a collection of terminal commands, while rqt is a set of visualization tools developed by ROS 2 based on the QT framework, specifically for robot development.

**Purpose**  
These tools facilitate program debugging, improve development efficiency, and enhance the user experience.


### Exercise ###
Create a node that publishes your wsuid, numerical value of your last three digits of your wsuid, and the division results of the last three digits of your wsuid divided by first three digits of your wsuid. The subscriber node will receive the message and print it out. 



## 2.2 Service Communication

**Scenario**

Service communication is another very porpuler communication model in ROS2. It is based on the request-response model, where one node sends a request to another node, and the latter responds with the result. Service communication is widely used in scenarios where a node needs to request data from another node, and the response is required to continue the operation.

For example, in a robot's navigation system, the path planning module may need to request the map data from the mapping module. The path planning module sends a request to the mapping module, and the mapping module responds with the map data. The path planning module then uses this data to plan the path.

Based on the previous example, the path planning module is the client, and the mapping module is the server. The client sends a request to the server, and the server responds with the result. The client and server nodes are decoupled, meaning they do not need to know each other's existence. The client node only needs to know the service name and the service type, while the server node only needs to know the service name and the service type.

**Concepts**

Service communication is based on the request-response model. In this model, a node sends a request to another node, and the latter responds with the result. The client node sends a request to the server node, and the server node processes the request and sends the response back to the client node.

<div style="text-align: center;">
    <img src="figs/2_3Service_Communicaiton_model_1.gif" alt="Service Communication Model" >
</div>

During service communication, the client node sends a request to the server node, and the server node processes the request and sends the response back to the client node. The client node then uses the response to continue its operation.

<div style="text-align: center;">
    <img src="figs/2_3Service_Communicaiton_model_2.gif" alt="Service Communication Model" >
</div>

**Purpose**

Service communication is typically used in scenarios where a node needs to request data from another node, and the response is required to continue the operation.

### Example of Service Communication ###

<div style = "text-align:center;">
    <img src = "figs/chapter_2_service_communication_demo.gif" alt="Service Communication Demo">
</div>


### Explaination of the Example ###

In this example, we will create two nodes: a server node and a client node. The server node will provide a service that returns the sum of two numbers, and the client node will send a request to the server node to calculate the sum.

### Edit the interface file ###

1. Create and edit the .srv file
2. Edit the package.xml file
3. Edit the CMakeLists.txt file
4. Test the custom service interface

1. Create and edit the .srv file
```plaintext
int32 num1
int32 num2
---
int32 sum
```

2. Edit the package.xml file

Since we will use the same package.xml, so wed don't need to modify it.
```xml
  <build_depend>rosidl_default_generators</build_depend>
  <depend>rclpy</depend>
  <depend>std_msgs</depend>
  <depend>base_interfaces_demo</depend>
```

3. Edit the CMakeLists.txt file
```plaintext

rosidl_generate_interfaces(${PROJECT_NAME}
  "msg/Student.msg"
  "srv/AddInts.srv"
)
```

4. Test
```bash

. install/setup.bash
ros2 interface show base_interfaces_demo/srv/AddInts

```

The output will be:

<div style = "text-align:center;">
    <img src = "figs/service_communication_srv_result.png" alt="Service Communication Demo">
</div>

5. Edit python files under the py02_service folder 

Define the server node

```python
"""
"""
 requirement: create a server
"""


# import nessesary modules
import rclpy
from rclpy.node import Node
from base_interfaces_demo.srv import AddInts    


class MinimalService(Node):

    def __init__(self):
        super().__init__('minimal_service')
        # create a server
        self.srv = self.create_service(AddInts, 'add_two_ints', self.add_two_ints_callback)
        self.get_logger().info('Server is ready to receive request...')

    def add_two_ints_callback(self, request, response):
        response.sum = request.num1 + request.num2
        self.get_logger().info('Incoming request\na: %d b: %d' % (request.num1, request.num2))
        return response
    

def main():
    # initialization 
    rclpy.init()
    # use the spin function 
    minimal_service = MinimalService()
    rclpy.spin(minimal_service)
    # shutdown the node
    rclpy.shutdown()


if __name__ == '__main__':
    main()
    
    
``` 

Define the client node

```python

"""
    requirements: edit the client, send two ints to the server, receive the sum of the two ints
"""

# import necessary packages
import sys
import rclpy
from rclpy.node import Node
from base_interfaces_demo.srv import AddInts


class MinimalClient(Node):
    
    def __init__(self):
        super().__init__('minimal_client_py')
        # create client
        self.cli = self.create_client(AddInts, 'add_ints')
        # wait for the service to be available
        while not self.cli.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('service not available, waiting again...')
        self.req = AddInts.Request()
        
    def send_request(self):
        # set request data
        self.req.num1 = int(sys.argv[1])
        self.req.num2 = int(sys.argv[2])
        # send request
        self.future = self.cli.call_async(self.req)
        
def main(args=None):
    # init rclpy
    rclpy.init(args=args)
    # create node
    minimal_client = MinimalClient()
    # send request
    minimal_client.send_request()
    # spin
    rclpy.spin_until_future_complete(minimal_client, minimal_client.future)
    try: 
        response = minimal_client.future.result()
    except Exception as e:
        minimal_client.get_logger().info(
            'Service call failed %r' % (e,))
    else:
        minimal_client.get_logger().info(
            'Result of add_two_ints: for %d + %d = %d' %
            (minimal_client.req.num1, minimal_client.req.num2, response.sum))
        
    # release resources
    rclpy.shutdown()
    
if __name__ == '__main__':
    main()
```



### Workflow of the Example ###

1. Create the server node
2. Create the client node
3. Edit the setup.py file
4. Compile
5. Run



## 2.3 Action Communication

**Scenario**

Regarding action communication, let's start by introducing it through the use case of navigation described below:

*A robot navigates to a specific target point. This process requires Node A to publish the target information, then Node B receives the request and controls the movement, ultimately responding with the target achievement status.*

At first glance, this seems like a task for service communication, as the requirement involves Node A sending the target, Node B executing it, and then returning a result. This is a typical request-response interaction model. However, using basic service communication alone presents a problem: navigation is a process, a time-consuming operation. If we use service communication, only at the end of the navigation would we get a response. During navigation, Node A would receive no feedback, potentially causing a "pseudo-freeze" in the program. This lack of control during the process can result in a poor user experience and logical flaws (such as being unable to handle navigation cancellation requests).

A more suitable solution would be to continuously provide feedback on the robot's current state throughout the navigation process, and only return the final result when navigation completes. In ROS, this implementation strategy is known as action communication.

**Concepts**

Action communication is suitable for long-running tasks. Structurally, action communication consists of three components: a goal, feedback, and a result. Functionally, it is similar to service communication, where the action client can send a request to the action server and receive the server's final response. However, action communication allows continuous feedback during the request-response process and enables the client to send a task cancellation request to the action server.

At the implementation level, action communication is built on top of topic and service communication. The goal transmission is a wrapper around service communication, as is the result retrieval, while continuous feedback is a wrapper around topic communication.
 
<div style = "text-align:center;">
    <img src = "figs/2_4_Action_Communicaiton_model_1.gif" alt="Service Communication Demo">
</div>



**Purpose**

Normally, action communication is used in scenarios where a long-running task requires continuous feedback and the ability to cancel the task.

### Example of Action Communication ###

<div style = "text-align:center;">
    <img src = "figs/chapter_2_demo_service_action_1_create_action_node_2.gif" alt="Service Communication Demo">
</div>


### Explaination of the Example ###

In this example, we will create two nodes: an action server node and an action client node. 



1. Create the package named ```base py03_action``` under the ```bash src``` folder
2. Create the action interface file
   1. Create .action file
        - Under package ```bash base_interfaces_demo```, create a new folder named ```action```, and create a new file named ```Progress.action```
        - ```bash 
            int64 num
            ---
            int64 sum
            ---
            float64 progress
            ```

        - Edit the package.xml file
          <buildtool_depend>rosidl_default_generators</buildtool_depend>
          <depend>action_msgs</depend>
          <member_of_group>rosidl_interface_packages</member_of_group>

        - Edit the CMakeLists.txt file
          ```bash
          rosidl_generate_interfaces(${PROJECT_NAME}
             "msg/Student.msg"
             "srv/AddInts.srv"
             "action/Progress.action"
            )  

         - colcon build --packages-select base_interfaces_demo
       - Test the custom action interface
         ```bash
         . install/setup.bash
         ros2 interface show base_interfaces_demo/action/Progress
         ```   
   2. Edit the package.xml, and setup.py file
   3. Compile
   4. Run

3. Edit python files under the py03_service folder 

Define the server node

```python
"""
requirement: Edit Action Server
"""

# import nessary libraries
import rclpy
import time
from rclpy.action import ActionServer
from rclpy.node import Node
from base_interfaces_demo.action import Progress



# create a node
class ProgressActionServer(Node):
    
    def __init__(self):
        super().__init__('progress_action_server')
        # create an action server
        self._action_server = ActionServer(
            self,
            Progress,
            'get_sum',
            self.execute_callback)
        self.get_logger().info('Action server is ready to receive requests...')
        
    def execute_callback(self, goal_handle):
        self.get_logger().info('Executing goal...')
        
        feedback_msg = Progress.Feedback()
        
        sum = 0 
        for i in range(1, goal_handle.request.num + 1):
            sum += i
            feedback_msg.progress = i / goal_handle.request.num
            self.get_logger().info('Contineous Feedback: %.2f' % feedback_msg.progress)
            goal_handle.publish_feedback(feedback_msg)
            time.sleep(1)

        goal_handle.succeed()
        result = Progress.Result()
        result.sum = sum
        self.get_logger().info('Done')
        
        return result
    
def main(args=None):
    # init rclpy
    rclpy.init(args=args)
    # create node
    Progress_action_server = ProgressActionServer()
    # spin
    rclpy.spin(Progress_action_server)
    # shutdown
    rclpy.shutdown()
    
if __name__ == '__main__':
    main()
    
``` 

Define the client node

```python
"""

    requirement: create an action client.
"""

# import necessary packages
import rclpy
from rclpy.action import ActionClient
from rclpy.node import Node
from base_interfaces_demo.action import Progress


# create a node
class ProgressActionClient(Node):
    
    def __init__(self):
        super().__init__('progress_action_client')
        # create an action client
        self._action_client = ActionClient(
            self,
            Progress,
            'get_sum')
    
    def send_goal(self, num):
        # send request
        goal_msg = Progress.Goal()
        goal_msg.num = num
        self._action_client.wait_for_server()
        self._send_goal_future = self._action_client.send_goal_async(goal_msg,
                                                                     feedback_callback=self.feedback_callback)
        
        self._send_goal_future.add_done_callback(self.goal_response_callback)
        
    def goal_response_callback(self, future):
        # handle goal response
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected...')
            return
        self.get_logger().info('Goal accepted...')
        
        self._get_result_future = goal_handle.get_result_async()
        self._get_result_future.add_done_callback(self.get_result_callback)
        
    def get_result_callback(self, future):
        # handle result
        result = future.result().result
        self.get_logger().info('Result: %d' % result.sum)
        rclpy.shutdown()
    
    def feedback_callback(self, feedback_msg):
        # handle feedback
        feedback = (int)(feedback_msg.feedback.progress * 100)
        self.get_logger().info('Current Progress: %d%%' % feedback)
        
    
def main(args=None):
    # init rclpy
    rclpy.init(args=args)
    # create node
    progress_action_client = ProgressActionClient()
    # send goal
    progress_action_client.send_goal(10)
    # spin
    rclpy.spin(progress_action_client)
    
    
if __name__ == '__main__':
    main()
```



## 2.4 Parameter Service

**Scenario**
In a robotic system, different functional modules may need to use some shared data. For example:

When implementing navigation, path planning is required, which mainly includes global path planning and local path planning. Global path planning involves designing an approximate route from the starting point to the target point, while local path planning generates a real-time path based on the vehicle’s current road conditions. Both types of path planning rely on the vehicle’s dimensions—such as length, width, and height.

So, how should this common data be stored and accessed in the program?

In this scenario, the parameter server can be used. Vehicle dimension data can be stored under one node, allowing other nodes to access and manipulate this data as needed.


**Concepts**

The parameter server is a communication model that facilitates data interaction between nodes in a shared manner. The node that stores parameters is called the parameter server, while nodes that retrieve parameters are called parameter clients. The interaction between the parameter client and the parameter server is request-response-based, and the implementation of parameter communication is essentially an additional layer of encapsulation over service communication.

**Purpose**

Parameter communication is typically used in scenarios where shared data is required between nodes. The parameter server can store shared data, and the parameter client can access and manipulate this data as needed.


